In [3]:
import numpy as np
import pandas as pd 

import pickle

from tqdm import tqdm

In [4]:
df_video = pd.read_orc('/kaggle/input/video-embeds-all/result_video_embeds_all.orc')

In [6]:
lst = [str(n) for n in range(0, 96)]
size = 1000000

for i in range(2,9):
    logs_one_day = pd.read_parquet(f'/kaggle/input/ohrenet-data-set-na-30-gb/logs_df_2024-07-2{i}.parquet')
    logs_one_day.dropna(inplace=True)

    unique_user_ids = np.random.choice(logs_one_day.user_id.unique(), size)

    logs_one_day = logs_one_day[logs_one_day.user_id.isin(unique_user_ids)]

    logs_one_day.drop(columns=['event_timestamp', 'region', 'city'], inplace=True)

    logs_one_day = logs_one_day.merge(df_video, on=['video_id'], how='left')

    logs_one_day['72'] = logs_one_day['72'].replace({0:0.01})
    logs_one_day.watchtime = logs_one_day.watchtime.replace({0:0.01})

    logs_one_day['watchtime_percent'] = logs_one_day['watchtime']/logs_one_day['72']
    logs_one_day = logs_one_day.drop(columns=['watchtime'])

    logs_one_day.watchtime_percent = logs_one_day.watchtime_percent.replace({np.inf:1})

    logs_one_day['watchtime_percent'] = np.where(logs_one_day['watchtime_percent']  > 1, 1, logs_one_day['watchtime_percent'])
    
    logs_one_day.iloc[:,2:-1] = logs_one_day.iloc[:,2:-1].mul(logs_one_day['watchtime_percent'], axis=0)
    users_vecs = logs_one_day.groupby('user_id')[lst].mean().reset_index()
    
    
    users_vecs.to_orc(f'users_vecs_2{i}.orc')

KeyboardInterrupt: 

In [33]:
# size = 10000
# unique_user_ids = np.random.choice(logs22.user_id.unique(), size)
# users_vecs_dict = {}

# for uid in tqdm(unique_user_ids):
#     user_logs = logs22.query(f'user_id == "{uid}"')
    
#     user_videos_list = list(user_logs.video_id.values)
#     user_watchtime_percent_list = list(user_logs.watchtime_percent.values)
    
#     values = [result_video_embeds_dict.get(key) for key in user_videos_list]
    
#     user_videos_vecs = np.array(values) * np.array(user_watchtime_percent_list)[:, np.newaxis]
    
#     user_vec = np.mean(user_videos_vecs, axis=0)
    
#     users_vecs_dict[uid] = user_vec